In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import matplotlib.dates as mdates
import re
import os
import datetime
import plotly
import warnings
import plotly.plotly as py
import plotly.graph_objs as go
# Offline mode
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.io as pio
from sklearn import preprocessing

In [16]:
#from google.colab import drive
#drive.mount('/content/drive')

In [17]:
#!ls "/content/drive/My Drive/Uni/Msc Comp Science/Year Two/Research/data"

In [18]:
# Set the global default size of matplotlib figures
plt.figure(figsize=(16,8))
plt.rc('figure', figsize=(16,8))


<Figure size 1152x576 with 0 Axes>

In [19]:
def show_dataframe_info(df):
    print(df.shape)
    print(list(df.columns.values))
    print('\n dataframe info: \n', df.info())
    print('\n dataframe column datatypes:\n', df.dtypes)

#show_dataframe_info()

def SetColor(x):
    if isinstance(x, float):
        if(x <= 12):
            return "green"
        elif(x > 12 and x <= 35.4):
            return "yellow"
        elif(x >35.4 and x <= 55.4 ):
            return "orange"
        elif(x >55.4 and x <= 150.4 ):
            return "red"
        elif(x > 150.4  and x <= 250.4 ):
            return "purple"
        elif(x > 250.4):
            return "maroon"
        #else:
            #return "black"
            
def remove_special_characters(original_string):
    cleaned_string = re.sub('[^A-Za-z0-9_-]+', '', original_string)
    return cleaned_string

def create_directory(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
def make_simple_line_chart(x_axis_data,y_axis_data, x_axis_label,y_axis_label, chart_title, saved_fig_full_filepath='chart'):
    fig = plt.figure(figsize=(24,12))
    plt.rcParams.update({'font.size':14, 'font.weight':'bold'})
    ax = fig.add_subplot(111)
    plt.plot(x_axis_data, y_axis_data, color='blue', marker='o', linestyle='solid')
    plt.title(chart_title, fontsize=20)
    plt.ylabel(y_axis_label, fontsize=20)
    plt.xlabel(x_axis_label,fontsize=20)
    
    hours = mdates.HourLocator(interval = 1)
    hour_format = mdates.DateFormatter('%H:%M')
    ax.xaxis.set_major_locator(hours)
    ax.xaxis.set_major_formatter(hour_format)
    
      ##coloring
    ax.axhspan(0, 12, facecolor='green')
    ax.axhspan(12, 35.4, facecolor='yellow')
    ax.axhspan(35.4, 55.4, facecolor='orange')
    ax.axhspan(55.4, 150.4, facecolor='red')
    ax.axhspan(150.4, 250.4, facecolor='purple')
    ax.axhspan(250.4, 500.4, facecolor='maroon')
    
    plt.show()
    fig.savefig(saved_fig_full_filepath + '.png')
    
    
def  generate_dates(number_of_days_to_subtract_from_currentdate):
    current_date = datetime.datetime.now()
    specified_date = current_date - datetime.timedelta(days =number_of_days_to_subtract_from_currentdate)
    return specified_date.strftime('%Y-%m-%d')
    
    

### Reading in greenness and airquality data for preprocessing


In [20]:
root_data_dir = 'data'
base_dir ='data\AirQuality\AirQo'
vi_base_dir ='data\Modis'

#airquality_data = pd.read_csv(os.path.join(base_dir,'Jinja_Road(NEMAHOUSE)','nema.csv'))
#vi_data = pd.read_csv(os.path.join(vi_base_dir,'jinja-road-nema-house-vi-2017-2019','jinja-road-nema-house-vi-2017-2019-MOD13A3-006-results.csv'))
#filepath = os.path.join(base_dir,'Jinja_Road(NEMAHOUSE)')

airquality_data = pd.read_csv(os.path.join(base_dir,'Rubaga_Kabusu','rubaga_kabusu.csv'))
vi_dataa = pd.read_csv(os.path.join(vi_base_dir,'rubaga-kabusu-vi-2017-2019','rubaga-kabusu-vi-2017-2019-MOD13A3-006-results.csv'))
filepath = os.path.join(base_dir,'Rubaga_Kabusu')

#airquality_data = pd.read_csv(os.path.join(base_dir,'Kireka','kireka.csv'))
#vi_dataa = pd.read_csv(os.path.join(vi_base_dir,'kireka-vi-2017-2019','kireka-vi-2017-2019-MOD13A3-006-results.csv'))
#filepath = os.path.join(base_dir,'Kireka')

airquality_data.head(10)

,created_at,entry_id,field1,field2,field3,field4,field5,field6,field7,field8,latitude,longitude,elevation,status
0,2019-03-28 10:24:23 EAT,1,77.08,83.93,70.38,75.70,0.000000,0.000000,3.78,"0.000000,0.000000, 0.00,0.00,0.00,0.00,26.00,...",NaN,NaN,NaN,NaN
1,2019-03-28 10:25:41 EAT,2,80.48,87.35,70.43,75.57,1000.000000,1000.000000,3.83,"1000.000000,1000.000000,1000000.00,-1.00,255.0...",NaN,NaN,NaN,NaN
2,2019-03-29 12:15:16 EAT,3,33.62,41.98,31.08,35.42,0.000000,0.000000,3.72,"0.000000,0.000000, 0.00,0.00,0.00,0.00,32.00,...",NaN,NaN,NaN,NaN
3,2019-03-29 12:16:34 EAT,4,29.87,34.98,31.13,34.20,1000.000000,1000.000000,3.93,"1000.000000,1000.000000,1000000.00,-1.00,255.0...",NaN,NaN,NaN,NaN
4,2019-03-29 12:17:50 EAT,5,28.87,31.40,30.10,33.17,1000.000000,1000.000000,4.01,"1000.000000,1000.000000,1000000.00,-1.00,255.0...",NaN,NaN,NaN,NaN
5,2019-03-29 12:19:07 EAT,6,38.63,46.82,39.02,46.07,1000.000000,1000.000000,4.02,"1000.000000,1000.000000,1000000.00,-1.00,255.0...",NaN,NaN,NaN,NaN
6,2019-03-29 12:20:31 EAT,7,28.42,33.82,24.18,25.45,1000.000000,1000.000000,4.05,"1000.000000,1000.000000,1000000.00,-1.00,255.0...",NaN,NaN,NaN,NaN
7,2019-03-29 12:21:48 EAT,8,45.83,56.00,42.35,50.23,0.297720,32.555157,4.06,"0.297720,32.555157,1000000.00,0.44,255.00,4294...",NaN,NaN,NaN,NaN
8,2019-03-29 12:23:05 EAT,9,34.65,40.22,33.48,39.20,0.297720,32.555157,4.07,"0.297720,32.555157,1205.70,0.44,4.00,451.00,34...",NaN,NaN,NaN,NaN
9,2019-03-29 12:24:21 EAT,10,39.28,47.67,31.68,36.53,0.297842,32.555035,4.07,"0.297842,32.555035,1194.50,0.07,5.00,220.00,34...",NaN,NaN,NaN,NaN


In [21]:

airquality_data.rename(columns={'field1':'Sensor1 PM2.5_CF_1_ug/m3','field2':'Sensor1 PM10_CF_1_ug/m3',
                                     'field3':'Sensor2PM2.5_CF_1_ug/m3', 'field4':'Sensor2 PM10_CF_1_ug/m3',
                                     'field5': 'Latitude', 'field6':'Longitude', 'field7':'Battery Voltage', 
                                     'field8':'GpsData'},inplace=True)

   
airquality_data['PM2.5(Ug/m3) '] = airquality_data[['Sensor1 PM2.5_CF_1_ug/m3', 'Sensor2PM2.5_CF_1_ug/m3']].mean(axis=1)
airquality_data['PM10(Ug/m3)'] = airquality_data[['Sensor1 PM10_CF_1_ug/m3', 'Sensor2 PM10_CF_1_ug/m3']].mean(axis=1)
    
 

In [22]:
airquality_data.head()

,created_at,entry_id,Sensor1 PM2.5_CF_1_ug/m3,Sensor1 PM10_CF_1_ug/m3,Sensor2PM2.5_CF_1_ug/m3,Sensor2 PM10_CF_1_ug/m3,Latitude,Longitude,Battery Voltage,GpsData,latitude,longitude,elevation,status,PM2.5(Ug/m3),PM10(Ug/m3)
0,2019-03-28 10:24:23 EAT,1,77.08,83.93,70.38,75.70,0.0,0.0,3.78,"0.000000,0.000000, 0.00,0.00,0.00,0.00,26.00,...",NaN,NaN,NaN,NaN,73.730,79.815
1,2019-03-28 10:25:41 EAT,2,80.48,87.35,70.43,75.57,1000.0,1000.0,3.83,"1000.000000,1000.000000,1000000.00,-1.00,255.0...",NaN,NaN,NaN,NaN,75.455,81.460
2,2019-03-29 12:15:16 EAT,3,33.62,41.98,31.08,35.42,0.0,0.0,3.72,"0.000000,0.000000, 0.00,0.00,0.00,0.00,32.00,...",NaN,NaN,NaN,NaN,32.350,38.700
3,2019-03-29 12:16:34 EAT,4,29.87,34.98,31.13,34.20,1000.0,1000.0,3.93,"1000.000000,1000.000000,1000000.00,-1.00,255.0...",NaN,NaN,NaN,NaN,30.500,34.590
4,2019-03-29 12:17:50 EAT,5,28.87,31.40,30.10,33.17,1000.0,1000.0,4.01,"1000.000000,1000.000000,1000000.00,-1.00,255.0...",NaN,NaN,NaN,NaN,29.485,32.285


In [23]:
airquality_data.tail()

,created_at,entry_id,Sensor1 PM2.5_CF_1_ug/m3,Sensor1 PM10_CF_1_ug/m3,Sensor2PM2.5_CF_1_ug/m3,Sensor2 PM10_CF_1_ug/m3,Latitude,Longitude,Battery Voltage,GpsData,latitude,longitude,elevation,status,PM2.5(Ug/m3),PM10(Ug/m3)
63894,2019-05-30 10:26:35 EAT,63895,61.88,74.37,52.10,62.33,0.297929,32.554970,4.17,"0.297929,32.554970,1226.40,0.19,9.00,84.00,30....",NaN,NaN,NaN,NaN,56.990,68.350
63895,2019-05-30 10:27:53 EAT,63896,60.10,72.77,51.02,62.20,0.297840,32.555004,4.18,"0.297840,32.555004,1213.00,0.05,10.00,87.00,30...",NaN,NaN,NaN,NaN,55.560,67.485
63896,2019-05-30 10:29:17 EAT,63897,45.72,58.37,40.78,48.15,0.297822,32.555012,4.17,"0.297822,32.555012,1212.20,0.11,9.00,100.00,30...",NaN,NaN,NaN,NaN,43.250,53.260
63897,2019-05-30 10:30:28 EAT,63898,35.57,41.85,37.78,42.93,0.297911,32.554955,4.18,"0.297911,32.554955,1225.30,0.20,10.00,127.00,3...",NaN,NaN,NaN,NaN,36.675,42.390
63898,2019-05-30 10:31:54 EAT,63899,39.07,49.55,35.78,41.60,0.297892,32.555019,4.18,"0.297892,32.555019,1223.60,0.21,8.00,96.00,31....",NaN,NaN,NaN,NaN,37.425,45.575


In [24]:
show_dataframe_info(airquality_data)

(63899, 16)
['created_at', 'entry_id', 'Sensor1 PM2.5_CF_1_ug/m3', 'Sensor1 PM10_CF_1_ug/m3', 'Sensor2PM2.5_CF_1_ug/m3', 'Sensor2 PM10_CF_1_ug/m3', 'Latitude', 'Longitude', 'Battery Voltage', 'GpsData', 'latitude', 'longitude', 'elevation', 'status', 'PM2.5(Ug/m3) ', 'PM10(Ug/m3)']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63899 entries, 0 to 63898
Data columns (total 16 columns):
created_at                  63899 non-null object
entry_id                    63899 non-null int64
Sensor1 PM2.5_CF_1_ug/m3    63899 non-null float64
Sensor1 PM10_CF_1_ug/m3     63899 non-null float64
Sensor2PM2.5_CF_1_ug/m3     63899 non-null float64
Sensor2 PM10_CF_1_ug/m3     63899 non-null float64
Latitude                    63899 non-null float64
Longitude                   63899 non-null float64
Battery Voltage             63899 non-null float64
GpsData                     63899 non-null object
latitude                    0 non-null float64
longitude                   0 non-null float64
elevatio

In [25]:
airquality_data.describe()

,entry_id,Sensor1 PM2.5_CF_1_ug/m3,Sensor1 PM10_CF_1_ug/m3,Sensor2PM2.5_CF_1_ug/m3,Sensor2 PM10_CF_1_ug/m3,Latitude,Longitude,Battery Voltage,latitude,longitude,elevation,status,PM2.5(Ug/m3),PM10(Ug/m3)
count,63899.000000,63899.000000,63899.000000,63899.000000,63899.000000,63899.000000,63899.000000,63899.000000,0.0,0.0,0.0,0.0,63899.000000,63899.000000
mean,31950.000000,52.447959,63.092592,49.585554,55.802232,0.610745,32.854724,4.158654,NaN,NaN,NaN,NaN,51.016757,59.447412
std,18446.196762,45.324713,53.212623,41.234693,43.060099,17.683736,17.116100,0.075546,NaN,NaN,NaN,NaN,43.085772,47.984041
min,1.000000,0.480000,0.850000,4.250000,4.480000,0.000000,0.000000,3.030000,NaN,NaN,NaN,NaN,3.090000,3.300000
25%,15975.500000,29.570000,35.420000,30.070000,33.550000,0.297842,32.554947,4.160000,NaN,NaN,NaN,NaN,29.787500,34.495000
50%,31950.000000,42.280000,53.970000,41.130000,48.950000,0.297869,32.554981,4.170000,NaN,NaN,NaN,NaN,41.725000,51.460000
75%,47924.500000,60.230000,73.770000,55.620000,65.530000,0.297898,32.555008,4.180000,NaN,NaN,NaN,NaN,57.842500,69.617500
max,63899.000000,971.630000,2021.000000,1198.200000,1402.630000,1000.000000,1000.000000,4.190000,NaN,NaN,NaN,NaN,1084.915000,1610.035000


Now that we have a general idea of the data set contents, we can dive deeper into each column. We'll be doing exploratory data analysis and cleaning data to setup 'features' we'll be using in our machine learning and deeplearning algorithms.

Plot a few features to get a better idea of each:



### Drop columns that shall not be used during analysis
- entry_id
- Battery Voltage
- Latitiude
- Longitude
- latitude
- longitude
- elevation
- status
- GpsData
- Sensor1 PM2.5_CF_1_ug/m3
- Sensor1 PM10_CF_1_ug/m3
- Sensor2PM2.5_CF_1_ug/m3'
- Sensor2 PM10_CF_1_ug/m3

In [26]:
airquality_data_of_interest = airquality_data.drop(['entry_id','Latitude', 'Longitude', 'Battery Voltage',
                                                    'latitude','longitude','elevation','status','GpsData',
                                                    'Sensor1 PM2.5_CF_1_ug/m3','Sensor1 PM10_CF_1_ug/m3',
                                                     'Sensor2PM2.5_CF_1_ug/m3','Sensor2 PM10_CF_1_ug/m3'], axis=1)

In [27]:
airquality_data_of_interest.shape
show_dataframe_info(airquality_data_of_interest)

(63899, 3)
['created_at', 'PM2.5(Ug/m3) ', 'PM10(Ug/m3)']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63899 entries, 0 to 63898
Data columns (total 3 columns):
created_at       63899 non-null object
PM2.5(Ug/m3)     63899 non-null float64
PM10(Ug/m3)      63899 non-null float64
dtypes: float64(2), object(1)
memory usage: 1.5+ MB

 dataframe info: 
 None

 dataframe column datatypes:
 created_at        object
PM2.5(Ug/m3)     float64
PM10(Ug/m3)      float64
dtype: object


### Convert data into timeseries dataframe with datetime being the index

In [28]:
#parsing the created_at field into a datetime object
airquality_data_of_interest["TimeStamp"] = pd.to_datetime(airquality_data_of_interest["created_at"])

is_utc_date = airquality_data_of_interest['created_at'].loc[0]
print(is_utc_date)

if "UTC" in is_utc_date:
    print('utc found')
    airquality_data_of_interest["TimeStamp"] = airquality_data_of_interest["TimeStamp"]+ datetime.timedelta(hours=3)

    
time_indexed_data = airquality_data_of_interest.set_index('TimeStamp')
time_indexed_data = time_indexed_data.drop(['created_at'], axis=1)

C:\Users\DELL\Anaconda3\envs\airquality_prediction\lib\site-packages\dateutil\parser\_parser.py:1204: UnknownTimezoneWarning:

tzname EAT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.



2019-03-28 10:24:23 EAT


In [29]:
time_indexed_data.head()

,PM2.5(Ug/m3),PM10(Ug/m3)
TimeStamp,,
2019-03-28 10:24:23,73.730,79.815
2019-03-28 10:25:41,75.455,81.460
2019-03-29 12:15:16,32.350,38.700
2019-03-29 12:16:34,30.500,34.590
2019-03-29 12:17:50,29.485,32.285


### Filter out outliers and remain with records with pm2.5 less than 500.4 and records with pm2.5 values  greater than zero 

In [30]:
time_indexed_data = time_indexed_data[time_indexed_data['PM2.5(Ug/m3)'] <= 500.4]
time_indexed_data = time_indexed_data[time_indexed_data['PM2.5(Ug/m3)'] > 0]
print(time_indexed_data.shape)
time_indexed_data.max()
time_indexed_data.min()

KeyError: 'PM2.5(Ug/m3)'

## Visualize the entire dataset pm2.5 timeseries

In [ ]:
y = time_indexed_data['PM2.5(Ug/m3)'].values
pm25 = go.Scatter(x=time_indexed_data.index,
                         y = time_indexed_data['PM2.5(Ug/m3)'].values
                 ) 

layout = go.Layout(title='PM2.5 Concentration ', xaxis=dict(title='Date'),
                   yaxis=dict(title='PM2.5 Concentration(ug/m3)'))

fig = go.Figure(data=[pm25], layout=layout)
iplot(fig)
pio.write_image(fig, os.path.join(filepath , 'pm25_concentration_timeseries.png'))

### Compute average hourly, daily, monthly concentration values (mean of the concentrations for each hour)
- parse the created_at column into datetime.
- set the created_at column to be the index for the dataframe.
- calculate the corresponding means for hourly, daily and monthly

In [ ]:
hourly_average_airquality_data_concentrations = time_indexed_data.resample('H').mean().round(2) 
hourly_maximum_airquality_data_concentrations  = time_indexed_data.resample('H').max().round(2)
hourly_minimum_airquality_data_concentrations  = time_indexed_data.resample('H').min().round(2)

daily_average_airquality_data_concentrations  = time_indexed_data.resample('D').mean().round(2)
daily_maximum_airquality_data_concentrations  = time_indexed_data.resample('D').max().round(2)
daily_minimum_airquality_data_concentrations  = time_indexed_data.resample('D').min().round(2)

monthly_average_airquality_data_concentrations  = time_indexed_data.resample('M').mean().round(2)


hourly_average_airquality_data_concentrations.head()

#drop nans
monthly_average_airquality_data_concentrations_analysis= monthly_average_airquality_data_concentrations.dropna(axis=0)
hourly_average_airquality_data_concentrations_analysis  = hourly_average_airquality_data_concentrations.dropna(axis=0)
daily_average_airquality_data_concentrations_analysis  =  daily_average_airquality_data_concentrations.dropna(axis=0)

In [ ]:
daily_average_airquality_data_concentrations

In [ ]:
print(daily_average_airquality_data_concentrations.size)
daily_average_missing_airquality_data = daily_average_airquality_data_concentrations[daily_average_airquality_data_concentrations.isnull()]

### Display information related to dataframe  after 24-hr-average(daily) and monthly average

In [ ]:
show_dataframe_info(daily_average_airquality_data_concentrations)

In [ ]:
daily_average_airquality_data_concentrations

In [ ]:
show_dataframe_info(daily_average_airquality_data_concentrations)

In [ ]:
show_dataframe_info(monthly_average_airquality_data_concentrations)

In [ ]:
monthly_average_airquality_data_concentrations

### filling in the missing values with the average of the value after and before 

In [ ]:
monthly_average_airquality_data_concentrations = monthly_average_airquality_data_concentrations.where(monthly_average_airquality_data_concentrations.notnull(), other=(monthly_average_airquality_data_concentrations.fillna(method='ffill')+monthly_average_airquality_data_concentrations.fillna(method='bfill'))/2)
daily_average_airquality_data_concentrations = daily_average_airquality_data_concentrations.where(daily_average_airquality_data_concentrations.notnull(), other=(daily_average_airquality_data_concentrations.fillna(method='ffill')+daily_average_airquality_data_concentrations.fillna(method='bfill'))/2)

### Visualize the daily  pm2.5 concentration timeseries

In [ ]:
print(daily_average_airquality_data_concentrations['PM2.5(Ug/m3)'].size)
y = daily_average_airquality_data_concentrations['PM2.5(Ug/m3)'].dropna().values
print(y.size)

In [ ]:
##remove nan for visualizations 
y = daily_average_airquality_data_concentrations['PM2.5(Ug/m3)'].dropna().values
pm25 = go.Scatter(x=daily_average_airquality_data_concentrations.dropna().index,
                  y = daily_average_airquality_data_concentrations['PM2.5(Ug/m3)'].dropna().values, 
                  mode='markers',
                  marker = dict(color=list(map(SetColor, y)),size=15)
                  
                 ) 

layout = go.Layout(title='Daily Average PM2.5 Concentration', xaxis=dict(title='Date'),
                   yaxis=dict(title='PM2.5 Concentration(ug/m3)'))

fig = go.Figure(data=[pm25], layout=layout)
iplot(fig)
pio.write_image(fig, os.path.join(filepath , 'daily_average_pm25_concentration.png'))


In [ ]:
average_daily_pm25 = go.Scatter(x=daily_average_airquality_data_concentrations['PM2.5(Ug/m3)'].dropna().index,
                         y = daily_average_airquality_data_concentrations['PM2.5(Ug/m3)'].dropna().values,
                         #mode='lines+markers',
                         #marker = dict(color=list(map(SetColor, y_average)), size = 17),
                         name='Average Daily PM2.5 Concentration'
                         #hoverinfo = 'text',
                         #text=[f'PM2.5 Concentration: {x:.2f} ug/m3' for x in daily_average_airquality_data_concentrations['PM2.5(Ug/m3)']]
                 )

layout = go.Layout(title='Daily PM2.5 Concentration', xaxis=dict(title='Date'),
                   yaxis=dict(title='PM2.5 Concentration(ug/m3)'))

fig = go.Figure(data=[average_daily_pm25], layout=layout)
iplot(fig)
pio.write_image(fig, os.path.join(filepath , 'daily_average_pm25_concentration_trend.png'))

### Visualize the monthly pm2.5 concentration

In [ ]:
average_monthly_pm25 = monthly_average_airquality_data_concentrations['PM2.5(Ug/m3)'].values
monthly_pm25 = go.Scatter(x=monthly_average_airquality_data_concentrations.index,
                         y = monthly_average_airquality_data_concentrations['PM2.5(Ug/m3)'].values,
                         mode='lines+markers',
                         marker = dict(color=list(map(SetColor, average_monthly_pm25)), size = 17),
                 ) 

layout = go.Layout(title='Monthly Average PM2.5 ', xaxis=dict(title='Date(UTC)'),
                   yaxis=dict(title='PM2.5 Concentration(ug/m3)'))

fig = go.Figure(data=[monthly_pm25], layout=layout)
iplot(fig)
pio.write_image(fig, os.path.join(filepath , 'monthly_average_pm25_concentration.png'))


In [ ]:
monthly_pm25_trend = go.Scatter(x=monthly_average_airquality_data_concentrations.index,
                         y = monthly_average_airquality_data_concentrations['PM2.5(Ug/m3)'].values,
                         #mode='lines+markers',
                        #marker = dict(color=list(map(SetColor, average_monthly_pm25)), size = 17),
                 ) 

layout = go.Layout(title='Monthly Average PM2.5 ', xaxis=dict(title='Date(UTC)'),
                   yaxis=dict(title='PM2.5 Concentration(ug/m3)'))

fig = go.Figure(data=[monthly_pm25_trend], layout=layout)
iplot(fig)
pio.write_image(fig, os.path.join(filepath , 'monthly_average_pm25_concentration_trend.png'))


### Display the concentrations
- hourly pm2.5
- daily  pm2.5
- monthly pm2.5

### Visualize the hourly pm2.5 concentration

In [ ]:
show_dataframe_info(hourly_average_airquality_data_concentrations)

In [ ]:
#hourly_airquality_data_concentrations['PM2.5(Ug/m3)'].max()

In [ ]:
hourly_pm25_trend = go.Scatter(x=hourly_average_airquality_data_concentrations.dropna().index,
                         y = hourly_average_airquality_data_concentrations['PM2.5(Ug/m3)'].dropna().values,
                 ) 

layout = go.Layout(title='Hourly Average PM2.5', xaxis=dict(title='Date'),
                   yaxis=dict(title='PM2.5 Concentration(ug/m3)'))

fig = go.Figure(data=[hourly_pm25_trend], layout=layout)
iplot(fig)
pio.write_image(fig,os.path.join(filepath , 'hourly_average_pm25_concentration_trend.png'))


In [ ]:
show_dataframe_info(hourly_average_airquality_data_concentrations)

In [ ]:
hourly_average_airquality_data_concentrations['hour'] = hourly_average_airquality_data_concentrations.index.hour
hourly_average_airquality_data_concentrations['Hour']= pd.to_timedelta(hourly_average_airquality_data_concentrations['hour'],unit='h')

hourly_average_airquality_data_concentrations_analysis['hour'] = hourly_average_airquality_data_concentrations_analysis.index.hour
hourly_average_airquality_data_concentrations_analysis['Hour']= pd.to_timedelta(hourly_average_airquality_data_concentrations_analysis['hour'],unit='h')

In [ ]:
current_date = generate_dates(0)
current_date

In [ ]:
show_dataframe_info(hourly_average_airquality_data_concentrations)

#### fill the missing hours with the average of the hours through the dataset

In [ ]:
cols = ['PM2.5(Ug/m3)','PM10(Ug/m3)']
original_hourly_data = hourly_average_airquality_data_concentrations.copy()
hourly_average_airquality_data_concentrations[cols] = hourly_average_airquality_data_concentrations[cols].fillna(hourly_average_airquality_data_concentrations.groupby(hourly_average_airquality_data_concentrations['Hour'])[cols].transform('mean').round(2))

In [ ]:
original_hourly_data.head(10)

In [ ]:
hourly_average_airquality_data_concentrations.head(10)

In [ ]:
average_pm25_concentration_for_each_hour_of_day = hourly_average_airquality_data_concentrations_analysis.groupby(['Hour']).mean().round(2) #.groups.keys()
average_pm25_concentration_for_each_hour_of_day

In [ ]:
current_date = datetime.datetime.now()
average_pm25_concentration_for_each_hour_of_day['Date'] = pd.to_datetime(current_date.strftime('%Y-%m-%d')) 
average_pm25_concentration_for_each_hour_of_day['Time(hrs)']= average_pm25_concentration_for_each_hour_of_day.index
average_pm25_concentration_for_each_hour_of_day['Time(hrs)']
average_pm25_concentration_for_each_hour_of_day[['Date','Time(hrs)']].dtypes
average_pm25_concentration_for_each_hour_of_day['TimeStamp']  = average_pm25_concentration_for_each_hour_of_day['Date']+ average_pm25_concentration_for_each_hour_of_day['Time(hrs)']
average_pm25_concentration_for_each_hour_of_day.head()

### Visualize the average of pm2.5 for each hour in the dataset

In [ ]:
y_axis_values = average_pm25_concentration_for_each_hour_of_day['PM2.5(Ug/m3)'].values
x_axis_values = average_pm25_concentration_for_each_hour_of_day['TimeStamp'].values
x_axis_label = 'Time(Hours)'
y_axis_label =  'PM2.5 Concentration(ug/m3)'
chart_title  =  'Average PM2.5 foreach Hour'
file_path    =   os.path.join(filepath , 'average_pm25_foreach_hour')

make_simple_line_chart(x_axis_values, y_axis_values, x_axis_label, y_axis_label, chart_title ,file_path)

In [ ]:
average_pm25_trend_for_each_hour = go.Scatter(x=average_pm25_concentration_for_each_hour_of_day['TimeStamp'].values,
                         y = average_pm25_concentration_for_each_hour_of_day['PM2.5(Ug/m3)'].values) 

layout = go.Layout(title='Average PM2.5 foreach Hour', xaxis=dict(title='Hour'),
                   yaxis=dict(title='PM2.5 Concentration(ug/m3)'))

fig = go.Figure(data=[average_pm25_trend_for_each_hour], layout=layout)
iplot(fig)
pio.write_image(fig, os.path.join(filepath , 'average_pm25_concentration_trend_for_each_hour.png'))

### Save the resampled pm2.5 concentration

In [ ]:
daily_average_airquality_data_concentrations.to_csv(os.path.join(filepath , 'daily_average_pm25_concentration.csv'))
monthly_average_airquality_data_concentrations.to_csv(os.path.join(filepath , 'monthly_average_pm25_concentration.csv'))
hourly_average_airquality_data_concentrations.to_csv(os.path.join(filepath , 'hourly_average_pm25_concentration.csv'))
average_pm25_concentration_for_each_hour_of_day.to_csv(os.path.join(filepath , 'average_pm25_concentration_for_each_hour_of_day.csv'))

daily_average_airquality_data_concentrations_analysis.to_csv(os.path.join(filepath , 'daily_average_pm25_concentration_analysis.csv'))
monthly_average_airquality_data_concentrations_analysis.to_csv(os.path.join(filepath , 'monthly_average_pm25_concentration_analysis.csv'))
hourly_average_airquality_data_concentrations_analysis.to_csv(os.path.join(filepath , 'hourly_average_pm25_concentration_analysis.csv'))

### Data analysis daily  airquality dataset

In [ ]:
show_dataframe_info(daily_average_airquality_data_concentrations)

### Show Histogram Distributions for the concentrations

In [ ]:
plt.figure(figsize=(16,8))
daily_average_airquality_data_concentrations.hist()
plt.savefig(os.path.join(filepath ,"daily_average_airquality_data_concentrations_historgram.png"))

In [ ]:
daily_average_airquality_data_concentrations.head(100)

In [ ]:
daily_average_airquality_data_concentrations.tail(10)

In [ ]:
#daily_airquality_data_concentrations.info()

### Plot the daily airquality data concentrations 

In [ ]:
values = daily_average_airquality_data_concentrations.values
# specify columns to plot
groups = [0,1,2]
i = 1
# plot each column

plt.figure()
for group in groups:
    plt.subplot(len(groups), 1, i)
    plt.plot(daily_average_airquality_data_concentrations.index,values[:, group])
    plt.title(daily_average_airquality_data_concentrations.columns[group], loc='right', y=0.5)
    plt.xlabel('Date')
    plt.ylabel(daily_average_airquality_data_concentrations.columns[group])
    i += 1
    
plt.savefig(os.path.join(filepath ,"daily_average_airquality_data_concentrations_linegraph.png"))
plt.show()

In [ ]:
values.shape

### Plotting the airquality timeseries

In [ ]:
airquality_pm25_values = values[:, 1]
plt.figure(figsize=(16,8))
plt.plot(daily_average_airquality_data_concentrations.index, airquality_pm25_values)
plt.title('PM2.5 Daily Concentrations')
plt.xlabel('Date')
plt.ylabel('PM2.5 Concentration(ug/m3)')
plt.savefig(os.path.join(filepath ,"daily_average_pm25_concentration_trend_linegraph.png"))
plt.show()

### Getting the date of the first and last record of the daily airquality dataset
This is essential to understand the date range for records from meteorology that shall be merged to this dataset

In [ ]:
## get the lower boundary date from the daily airquality dataset
daily_average_airquality_data_concentrations.iloc[:1,:]
t= daily_average_airquality_data_concentrations.iloc[:1,:].index.values[0]
t = t.astype(datetime.datetime)
z= pd.to_datetime(t)
daily_average_airquality_data_lower_boundary_date = z.strftime('%Y-%m-%d')

print(daily_average_airquality_data_lower_boundary_date)
## get the upper boundary date from the daily airquality dataset
t = daily_average_airquality_data_concentrations.iloc[-1:,:].index.values[0]
t = t.astype(datetime.datetime)
z= pd.to_datetime(t)
daily_average_airquality_data_upper_boundary_date = z.strftime('%Y-%m-%d')
print(daily_average_airquality_data_upper_boundary_date)



### Getting the datetime of the first and last record of the hourly airquality dataset

In [ ]:
## get the lower boundary date from the hourly airquality dataset
hourly_average_airquality_data_concentrations.iloc[:1,:]
h= hourly_average_airquality_data_concentrations.iloc[:1,:].index.values[0]
h = h.astype(datetime.datetime)
y= pd.to_datetime(h)
hourly_average_airquality_data_lower_boundary_date = y.strftime('%Y-%m-%d %H:%M:%S')
  
print(hourly_average_airquality_data_lower_boundary_date)
## get the upper boundary date from the hourly airquality dataset
h = hourly_average_airquality_data_concentrations.iloc[-1:,:].index.values[0]
h = h.astype(datetime.datetime)
y= pd.to_datetime(h)
hourly_average_airquality_data_upper_boundary_date = y.strftime('%Y-%m-%d %H:%M:%S')
print(hourly_average_airquality_data_upper_boundary_date)

### Analysing the relationship among the airpollutant concentrations in relation to pm2.5
- use scatter plots to determine the extent of correlation and sense of correlation between pm2.5 and other air pollutant concentrations i.e pm1, pm10, co, so2.

- this can be acomplished used sns pairplot or matplot pair plot for each pair of variables.

In [ ]:
import seaborn as sns;
sns.pairplot(daily_average_airquality_data_concentrations)
plt.savefig(os.path.join(filepath ,"daily_average_airquality_data_concentrations_pairplot_corr.png"))
plt.show()

In [ ]:
daily_average_airquality_data_concentrations.corr()

### Read in the Meteorological data for preprocessing it
- read in dataset
- display first 10 records
- display last 10 records
- display dataset summary i.e shape, column names, info and datatypes for each column
- generate and display descriptive statistics that summarize the central tendency, dispersion and shape of a dataset’s  distribution

In [ ]:
daily_meteorological_data = pd.read_csv('data/Met/tahmo_Makerere_Synopticdata_daily.csv')
monthly_meteorological_data = pd.read_csv('data/Met/tahmo_Makerere_Synopticdata_monthly.csv')
hourly_meteorological_data = pd.read_csv('data/Met/tahmo_Makerere_Synopticdata_hourly.csv')
daily_tahmo_unma_rainfall_comparison_data = pd.read_csv('data/Met/tahmo_unma_rainfall_comparison_daily.csv')
daily_meteorological_data.head(10)

### comparison of UNMA and tahmo daily rainfall dataset

In [ ]:
show_dataframe_info(daily_tahmo_unma_rainfall_comparison_data)
daily_tahmo_unma_rainfall_comparison_data_without_missing_records =  daily_tahmo_unma_rainfall_comparison_data.dropna()
daily_tahmo_unma_rainfall_comparison_data_without_missing_records["Date"] = pd.to_datetime(daily_tahmo_unma_rainfall_comparison_data_without_missing_records["Date"])
daily_tahmo_unma_rainfall_comparison_data_without_missing_values = daily_tahmo_unma_rainfall_comparison_data_without_missing_records.set_index('Date')
print(daily_tahmo_unma_rainfall_comparison_data.head(3))
show_dataframe_info(daily_tahmo_unma_rainfall_comparison_data_without_missing_values)

In [ ]:
daily_tahmo_unma_rainfall_comparison_data_without_missing_values.head()

In [ ]:
show_dataframe_info(daily_tahmo_unma_rainfall_comparison_data)

### Correlation between rainfall observations from unma and tahmo

In [ ]:
daily_tahmo_unma_rainfall_comparison_data_correlations= daily_tahmo_unma_rainfall_comparison_data_without_missing_values.corr()
daily_tahmo_unma_rainfall_comparison_data_correlations.to_csv(os.path.join(filepath, 'daily_tahmo_unma_rainfall_comparison_data_correlations.csv'))
daily_tahmo_unma_rainfall_comparison_data_correlations

### Line graphs showing comparison of rainfall dataset from unma and tahmo

In [ ]:
    fig = plt.figure(figsize=(16,6))
    plt.rcParams.update({'font.size':12, 'font.weight':'bold'})
    plt.plot(daily_tahmo_unma_rainfall_comparison_data_without_missing_values.index, 
             daily_tahmo_unma_rainfall_comparison_data_without_missing_values['UNMA'], 
             color='olive',  linestyle='solid', label='UNMA RAINFALL OBSERVATIONS')
    plt.plot(daily_tahmo_unma_rainfall_comparison_data_without_missing_values.index, 
             daily_tahmo_unma_rainfall_comparison_data_without_missing_values['Tahmo'], 
             color='chocolate', linestyle='solid', label='Tahmo RAINFALL OBSERVATIONS')
   
    #marker='o', marker='x',
    chart_title =  'Comparison of Daily Rainfall Observations between UNMA and Tahmo '
    plt.title(chart_title,fontsize=20)
    plt.ylabel('RAINFALL OBSERVATIONS', fontsize=20)
    plt.xlabel('Date', fontsize=20)
    #plt.margins(0.1)
    plt.legend()
    plt.show()
    path_to_save_file = os.path.join('data','Met')
    fig.savefig(os.path.join( path_to_save_file , 'daily_tahmo_unma_rainfall_comparison_data.png'))

In [ ]:
daily_meteorological_data.tail()

In [ ]:
show_dataframe_info(daily_meteorological_data)

In [ ]:
daily_meteorological_data.describe()

### Drop columns from daily,hourly and monthly meteorological dataset (tahmo) that shall not be used during analysis

### daily
- site 
- name
- max temperature
- min temperature

### hourly
- site 
- name

### monthly
- site 
- name
- min windspeed(m/s)
- max windspeed(m/s)

In [ ]:
daily_meteorological_data_of_interest = daily_meteorological_data.drop(
    ['site','name','min temperature (oC)','max temperature (oC)'], axis=1)

hourly_meteorological_data_of_interest = hourly_meteorological_data.drop(['site','name'], axis=1)

monthly_meteorological_data_of_interest = monthly_meteorological_data.drop(
    ['site','name','min temperature (oC)','max temperature (oC)','max windspeed (m/s)','min windspeed (m/s)'], axis=1)

In [ ]:
daily_meteorological_data_of_interest.head()

In [ ]:
show_dataframe_info(daily_meteorological_data_of_interest)

In [ ]:
hourly_meteorological_data.head()

In [ ]:
show_dataframe_info(hourly_meteorological_data)

In [ ]:
monthly_meteorological_data_of_interest.head()

In [ ]:
show_dataframe_info(monthly_meteorological_data)

###  Set the day column to be the index for the daily meteorological dataset
- parse the  day column into datetime.
- set the  day column to be the index for the dataframe.

### set the dateTimeUTC to be the index of the hourly meteorological dataset
- parse the  dateTimeUTC column into datetime.
- set the  dateTimeUTC column to be the index for the dataframe.
- change the hourly data from UTC to EAT(Nairobi) ugandanda time.

In [ ]:
daily_meteorological_data_of_interest["day"] = pd.to_datetime(daily_meteorological_data_of_interest["day"])
daily_meteorological_data_values  = daily_meteorological_data_of_interest.set_index('day')


monthly_meteorological_data_of_interest["month"] = pd.to_datetime(monthly_meteorological_data_of_interest["month"])
monthly_meteorological_data_values = monthly_meteorological_data_of_interest.set_index('month')

hourly_meteorological_data_of_interest["dateTimeUTC"] = pd.to_datetime(hourly_meteorological_data_of_interest["dateTimeUTC"])
hourly_meteorological_data_of_interest["dateTime"] = hourly_meteorological_data_of_interest["dateTimeUTC"] + datetime.timedelta(hours=3)
hourly_meteorological_data_values = hourly_meteorological_data_of_interest.set_index('dateTime')
hourly_meteorological_data_values = hourly_meteorological_data_values.drop(['dateTimeUTC'], axis=1)


### Data Analysis on Monthly Meteorological data

In [ ]:
monthly_meteorological_data_values.head()

In [ ]:
show_dataframe_info(monthly_meteorological_data_values)

### Show histogram distributions for the monthly meteorological data

In [ ]:
monthly_meteorological_data_values.hist()
plt.savefig(os.path.join(filepath ,"monthly_meteorological_data_historgram.png"))
plt.show()

### Data analysis on daily meteorological dataset

In [ ]:
show_dataframe_info(daily_meteorological_data_values)

### Show histogram distributions for the daily meteorology data

In [ ]:
daily_meteorological_data_values.hist()
plt.savefig(os.path.join(filepath ,"daily_meteorological_data_historgram.png"))
plt.show()

In [ ]:
daily_meteorological_data_values.head(10)

### Show histogram distributions for the hourly meteorological data

In [ ]:
hourly_meteorological_data_values.hist()
plt.savefig(os.path.join(filepath ,"hourly_meteorological_data_historgram.png"))
plt.show()

### One Hot Encoding Categorical features (WindDirection)

In [ ]:
daily_meteorological_data_values_onehot =  daily_meteorological_data_values.copy()
daily_meteorological_data_values_onehot = pd.get_dummies( daily_meteorological_data_values_onehot, columns=['predominant winddirection'], prefix = ['winddirection'])

daily_meteorological_data_values_onehot.head()
show_dataframe_info(daily_meteorological_data_values_onehot)

In [ ]:
hourly_meteorological_data_values_onehot =  hourly_meteorological_data_values.copy()
hourly_meteorological_data_values_onehot = pd.get_dummies( hourly_meteorological_data_values_onehot, columns=['winddirection'], prefix = ['winddirection'])

hourly_meteorological_data_values_onehot.head()
show_dataframe_info(hourly_meteorological_data_values_onehot)

### Obtaining Meteorology data in the same date range as airquality data

In [ ]:
daily_meteorological_data_in_same_daterange_with_daily_airquality_data= daily_meteorological_data_values_onehot[daily_average_airquality_data_lower_boundary_date : daily_average_airquality_data_upper_boundary_date]
hourly_meteorological_data_in_same_daterange_with_daily_airquality_data= hourly_meteorological_data_values_onehot[hourly_average_airquality_data_lower_boundary_date : hourly_average_airquality_data_upper_boundary_date]

In [ ]:
daily_meteorological_data_in_same_daterange_with_daily_airquality_data.head()

In [ ]:
show_dataframe_info(daily_meteorological_data_in_same_daterange_with_daily_airquality_data)

#### Handle missing data in daily meteorology data by filling NaN with the Mean of each column

In [ ]:
daily_meteorological_data_with_out_missing_values =daily_meteorological_data_in_same_daterange_with_daily_airquality_data
#daily_meteorological_data_with_out_missing_values.fillna(daily_meteorological_data_with_out_missing_values.mean(),inplace=True)
show_dataframe_info(daily_meteorological_data_with_out_missing_values)

### Plot the daily meteorology data values for the time series

In [ ]:
meteorological_values = daily_meteorological_data_with_out_missing_values.values
# specify columns to plot
groups = [0,1,2, 3, 4]
i = 1
# plot each column

plt.figure()
for group in groups:
    #plt.subplot(len(groups), 1, i)
    plt.plot(daily_meteorological_data_with_out_missing_values.index,meteorological_values[:, group])
    plt.title(daily_meteorological_data_with_out_missing_values.columns[group], loc='right', y=0.8)
    plt.xlabel('Date')
    plt.ylabel(daily_meteorological_data_with_out_missing_values.columns[group])
    #i += 1
    plt.show()

### Vegetation Index (greenness Dataset)

In [ ]:
show_dataframe_info(vi_data)

In [ ]:
vi_data_of_interest.head()

In [ ]:
vi_mask = vi_data.dtypes == np.float64
vi_columns = vi_data.columns[vi_mask]
#vi_data_of_interest = vi_data[]
vi_data_of_interest = vi_data[vi_columns]
vi_data_of_interest['TimeStamp'] = vi_data['Date']
vi_data_of_interest = vi_data_of_interest.drop(
    ['Latitude','Longitude','MOD13A3_006_Sample_X_1km','MOD13A3_006_Line_Y_1km','MOD13A3_006__1_km_monthly_VI_Quality'], axis=1)
#

In [ ]:
vi_data_of_interest.head()

In [ ]:
 vi_data_of_interest.rename(columns={'MOD13A3_006__1_km_monthly_EVI':'EVI','MOD13A3_006__1_km_monthly_NDVI':'NDVI'},inplace=True)

In [ ]:
monthly_vi_data = vi_data_of_interest.set_index('TimeStamp')
monthly_vi_data_copy = monthly_vi_data.copy()
monthly_vi_data

### Analysing Relationship between PM2.5 and Greenness ( merging vi dataset and airquality dataset)

In [ ]:
monthly_average_airquality_data_concentrations.head()
dates = monthly_average_airquality_data_concentrations.index
monthly_average_airquality_data_concentrations['MonthOfYear'] = pd.to_datetime(dates).strftime('%m-%Y') 
monthly_average_airquality_data_concentrations.head()

vi_dates = monthly_vi_data.index
monthly_vi_data['MonthOfYear'] = pd.to_datetime(vi_dates).strftime('%m-%Y') 

#print(monthly_average_airquality_data_concentrations.head())
#print(monthly_vi_data.head())

meteorological_monthly_dates = monthly_meteorological_data_values.index
monthly_meteorological_data_values['MonthOfYear'] = pd.to_datetime(meteorological_monthly_dates).strftime('%m-%Y') 
monthly_meteorological_data_values.head()


In [ ]:
## get the lower boundary date from the monthly airquality dataset
monthly_average_airquality_data_concentrations.iloc[:1,:]
first_record= monthly_average_airquality_data_concentrations.iloc[:1,:].index.values[0]
first_record = first_record.astype(datetime.datetime)
x= pd.to_datetime(first_record)
monthly_average_airquality_data_lower_boundary_date = x.strftime('%m-%Y')
 
print(monthly_average_airquality_data_lower_boundary_date)
## get the upper boundary date from the monthly airquality dataset
last_record = monthly_average_airquality_data_concentrations.iloc[-1:,:].index.values[0]
last_record = last_record.astype(datetime.datetime)
y = pd.to_datetime(last_record)
monthly_average_airquality_data_upper_boundary_date = y.strftime('%m-%Y')
print(monthly_average_airquality_data_upper_boundary_date)


monthly_combined_airquality_vi_dataset= pd.merge(monthly_average_airquality_data_concentrations, monthly_vi_data, on='MonthOfYear')
merged_monthly_airquality_greenness_meteorological_dataset = pd.merge(monthly_combined_airquality_vi_dataset, monthly_meteorological_data_values, on='MonthOfYear')
merged_monthly_airquality_vi_dataset = monthly_combined_airquality_vi_dataset.set_index('MonthOfYear')
merged_monthly_airquality_greenness_meteorological_dataset = merged_monthly_airquality_greenness_meteorological_dataset.set_index('MonthOfYear')

print(merged_monthly_airquality_vi_dataset.head())
merged_monthly_airquality_greenness_meteorological_dataset.head()

In [ ]:
vi_airquality_correlations= merged_monthly_airquality_vi_dataset.corr()
vi_airquality_correlations.to_csv(os.path.join(filepath, 'vi_airquality_correlations.csv'))
vi_airquality_correlations

In [ ]:
merged_monthly_airquality_vi_dataset_historgram = merged_monthly_airquality_vi_dataset.hist()
plt.savefig(os.path.join(filepath ,"merged_monthly_airquality_vi_dataset_historgram.png"))
plt.show()
#merged_monthly_airquality_vi_dataset_historgram_figure = merged_monthly_airquality_vi_dataset_historgram[0].get_figure() 
#merged_monthly_airquality_vi_dataset_historgram_figure.savefig(os.path.join(filepath ,"merged_monthly_airquality_vi_dataset_historgram.png"))

In [ ]:
merged_monthly_airquality_vi_sns_plot= sns.pairplot(merged_monthly_airquality_vi_dataset)
plt.show()
merged_monthly_airquality_vi_sns_plot.savefig(os.path.join(filepath ,"merged_monthly_airquality_vi_sns_plot.png"))

In [ ]:
merged_monthly_pm25_ndvi = merged_monthly_airquality_vi_dataset[['PM2.5(Ug/m3)', 'NDVI']].copy()
merged_monthly_pm25_ndvi.head()
corr_plot = sns.regplot(merged_monthly_pm25_ndvi['PM2.5(Ug/m3)'], merged_monthly_pm25_ndvi['NDVI'])
corr_plot_figure = corr_plot.get_figure()    
corr_plot_figure.savefig(os.path.join(filepath ,"corr_pm25_vi_reg_plot.png"))

In [ ]:
merged_monthly_pm25_ndvi.corr(method='pearson')

In [ ]:
pm25_ndvi_sns_plot= sns.pairplot(merged_monthly_pm25_ndvi)
plt.show()
pm25_ndvi_sns_plot.savefig(os.path.join(filepath ,"monthly_pm25_vi_sns_plot.png"))

In [ ]:
fig_corr_matrix = plt.figure(figsize=(12,6))
ax_fig_corr_matrix = fig_corr_matrix.add_subplot(111)
cax = ax_fig_corr_matrix.matshow(merged_monthly_pm25_ndvi.corr())
plt.xticks(range(len(merged_monthly_pm25_ndvi.columns)), merged_monthly_pm25_ndvi.columns)
plt.yticks(range(len(merged_monthly_pm25_ndvi.columns)),merged_monthly_pm25_ndvi.columns)
fig_corr_matrix.colorbar(cax)
plt.show()

fig_corr_matrix.savefig(os.path.join(filepath, 'monthly_pm25_vi_corr_matrix.png'))

### Combining the daily airquality dataset , daily meteorological dataset and greenness(VI) dataset
 - The vegetation indices are sampled on monthly basis, we upsampled them to get values for daily vi data

In [ ]:
daily_airquality_data =  daily_average_airquality_data_concentrations
print(daily_airquality_data.shape)
daily_timestamp = daily_airquality_data.index.values
print(len(daily_timestamp))
daily_airquality_data["TimeStamp"] = daily_timestamp
print(daily_airquality_data.columns)
daily_airquality_data["TimeStamp"] = pd.to_datetime(daily_airquality_data["TimeStamp"])
print(daily_airquality_data.shape)

# month and year for merging with vi data
daily_airquality_data['MonthOfYear'] = pd.to_datetime(daily_timestamp).strftime('%m-%Y') 
daily_airquality_data.head()

In [ ]:
daily_airquality_data.head()

In [ ]:
daily_meteorology_data =  daily_meteorological_data_with_out_missing_values
print(daily_meteorology_data.shape)
timestamp = daily_meteorology_data.index.values
print(len(timestamp))
daily_meteorology_data["TimeStamp"] = timestamp
print(daily_meteorology_data.columns)
daily_meteorology_data["TimeStamp"] = pd.to_datetime(daily_meteorology_data["TimeStamp"])
print(daily_meteorology_data.shape)

In [ ]:
daily_meteorology_data.head()

In [ ]:
daily_meteorology_data

## merging vi  data to daily airquality data
   - to comeup with daily airquality data containing vi data

In [ ]:
daily_combined_airquality_vi_dataset= pd.merge(daily_airquality_data, monthly_vi_data, on='MonthOfYear')

In [ ]:
daily_combined_airquality_vi_dataset.head(3)

In [ ]:
daily_combined_airquality_vi_dataset.columns

In [ ]:
daily_merged_airquality_vi_dataset_of_interest= daily_combined_airquality_vi_dataset.drop(['MonthOfYear'], axis=1)
daily_merged_airquality_vi_dataset = daily_merged_airquality_vi_dataset_of_interest.set_index('TimeStamp')
daily_merged_airquality_vi_dataset.head()

In [ ]:
daily_merged_airquality_vi_dataset.tail()

In [ ]:
show_dataframe_info(daily_merged_airquality_vi_dataset)

In [ ]:
print(daily_meteorology_data.shape)

In [ ]:
daily_combined_dataset= pd.merge( daily_merged_airquality_vi_dataset,daily_meteorology_data, on='TimeStamp')

In [ ]:
daily_combined_dataset.shape

In [ ]:
show_dataframe_info(daily_combined_dataset)

In [ ]:
daily_combined_dataset.head(20)

### Remove the time column from combined dataset
- make timestamp  the index values

In [ ]:
merged_daily_airquality_greenness_meteorological_dataset  = daily_combined_dataset.set_index('TimeStamp')
merged_daily_airquality_greenness_meteorological_dataset.head()

###  merging vi  data to hourly airquality and hourly meteorological data
   - to comeup with hourly airquality data containing vi data

In [ ]:
hourly_meteorological_data_in_same_daterange_with_daily_airquality_data.head(4)

In [ ]:
show_dataframe_info(hourly_meteorological_data_values)

In [ ]:
hourly_average_airquality_data_concentrations.head()

In [ ]:
show_dataframe_info(hourly_average_airquality_data_concentrations)

In [ ]:
hourly_airquality_data =  hourly_average_airquality_data_concentrations
print(hourly_airquality_data.shape)
hourly_timestamp = hourly_airquality_data.index.values
print(len(hourly_timestamp))
hourly_airquality_data["TimeStamp"] = hourly_timestamp
print(hourly_airquality_data.columns)
hourly_airquality_data["TimeStamp"] = pd.to_datetime(hourly_airquality_data["TimeStamp"])
print(hourly_airquality_data.shape)

# month and year for merging with vi data
hourly_airquality_data['MonthOfYear'] = pd.to_datetime(hourly_timestamp).strftime('%m-%Y') 
hourly_airquality_data.head()

In [ ]:
hourly_meteorology_data =  hourly_meteorological_data_in_same_daterange_with_daily_airquality_data
print(hourly_meteorology_data.shape)
hourly_timestamp = hourly_meteorology_data.index.values
#print(len(hourly_timestamp))
hourly_meteorology_data["TimeStamp"] = hourly_timestamp
#print(hourly_meteorology_data.columns)
hourly_meteorology_data["TimeStamp"] = pd.to_datetime(hourly_meteorology_data["TimeStamp"])
print(hourly_meteorology_data.shape)

In [ ]:
hourly_combined_airquality_vi_dataset= pd.merge(hourly_airquality_data, monthly_vi_data, on='MonthOfYear')
hourly_merged_airquality_vi_dataset_of_interest = hourly_combined_airquality_vi_dataset.drop(['MonthOfYear'], axis=1)
hourly_merged_airquality_vi_dataset = hourly_merged_airquality_vi_dataset_of_interest.set_index('TimeStamp')
hourly_merged_airquality_vi_dataset.head()
#hourly_combined_airquality_vi_dataset.head(3)

In [ ]:
hourly_combined_dataset= pd.merge(hourly_merged_airquality_vi_dataset,hourly_meteorology_data, on='TimeStamp')
merged_hourly_airquality_greenness_meteorological_dataset  = hourly_combined_dataset.set_index('TimeStamp')
merged_hourly_airquality_greenness_meteorological_dataset.head()

In [ ]:
#show_dataframe_info(hourly_combined_dataset)
merged_hourly_airquality_greenness_meteorological_dataset = merged_hourly_airquality_greenness_meteorological_dataset.drop(['Hour'], axis=1)
merged_hourly_airquality_greenness_meteorological_dataset.head()

### Save the combined datasets to the respective folder.

In [ ]:
merged_monthly_airquality_greenness_meteorological_dataset.to_csv(os.path.join(filepath , 'merged_monthly_airquality_greenness_meteorological_dataset.csv'))
merged_daily_airquality_greenness_meteorological_dataset.to_csv(os.path.join(filepath , 'merged_daily_airquality_greenness_meteorological_dataset.csv'))
merged_hourly_airquality_greenness_meteorological_dataset.to_csv(os.path.join(filepath , 'merged_hourly_airquality_greenness_meteorological_dataset.csv'))

### Feature Selection